In [1]:
import pandas as pd
import numpy as np

df = pd.read_parquet('../data/open-stax-texts-terms.parquet')

In [11]:
def convert_to_jsonl(dataframe, labels_column='all_term_offsets'):
    (dataframe[['url', 'text', labels_column]]
     .rename(columns={'url':'name',
                      labels_column:'labels'
                     })
     .to_json(f'../assets/openstax_{labels_column}.jsonl',
              orient='records',
              lines=True)
    )

convert_to_jsonl(df, labels_column='all_term_offsets')

In [12]:
convert_to_jsonl(df, labels_column='bold_term_offsets')
convert_to_jsonl(df, labels_column='no_bold_term_offsets')